In [ ]:
import json
from pandas import DataFrame
from pymatgen import MPRester, Structure
from mpcontribs.client import load_client

In [ ]:
with open('/Users/patrick/Downloads/log_kvrh', 'r') as f:
    data = json.load(f)
df = DataFrame(data['data'], columns=data['columns'])

In [ ]:
project = 'matbench_v1'
client = load_client('your-api-key-here')
mpr = MPRester()

In [ ]:
client.projects.get_entry(pk=project).result()

In [ ]:
for idx, (s, kvrh) in enumerate(zip(df['structure'], df['log10(K_VRH)'])):
    structure = Structure.from_dict(s)
    identifier = mpr.find_structure(structure)[0]
    contrib = {'project': project, 'identifier': identifier, 'data': {'K|VRH': kvrh}}
    cid = client.contributions.create_entry(contribution=contrib).result()['id']
    sdct = dict(contribution=cid, name=structure.formula.replace(' ', ''))
    sdct.update(structure.as_dict())
    try:
        sid = client.structures.create_entry(structure=sdct).result()['id']
    except Exception as ex:
        print(ex)
        print(sdct)
        client.contributions.delete_entry(pk=cid).result()
        break
    print(idx, cid)
    if idx > 15:
        break

In [ ]:
contribs = client.contributions.get_entries(project=project, _fields=['id'], _limit=100).result()

In [ ]:
for idx, contrib in enumerate(contribs['data']):
    client.contributions.delete_entry(pk=contrib['id']).result()
    print(idx)